# Prediction of craft beer price in polish pubs

Aim of this project is to predict pub price for craft beer.

Scope of project are pubs which are placing their pricelist on ontap.pl portal - polish aggregator for pubs beer

## Source data

Data has been scrapped from ontap.pl site

### Basic imports

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import numpy as np
import pandas as pd
import re

# plots
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

### Logger setup

In [2]:
import datetime
import logging
logger_scrapping = logging.getLogger(__name__)
logger_scrapping.setLevel(logging.INFO)
fh = logging.FileHandler('scrapping.log', mode='w', encoding='utf8')
logger_scrapping.addHandler(fh)

### Dictionaries

In [3]:
breweries = {
    'amber' : 'browar amber',
    'bracki browar zamkowy w cieszynie' : 'browar zamkowy cieszyn',
    'alebrowar' : 'alebrowar',
    'alebrowar' : 'alebrowar',
    'nocny kochanek / alebrowar' : 'alebrowar',
    'browar pinta' : 'browar browar pinta',
    'beer bros.' : 'beer bros',
    'beer bross' : 'beer bros',
    'beerbross' : 'beer bros',
    'brekreacja' : 'brokreacja', 
    'browar pinta' : 'browar pinta',
    'cieszyn' : 'browar zamkowy cieszyn',
    'cześć brat' : 'cześć brat!',
    'dear bear' : 'deer bear',
    'deer beer' :  'deer bear', 
    'fourpure' : 'fourpure brewing co.',
    'fullers (uk)' : "fuller's",
    'funky fluid / white whale' : 'funky fluid',
    'nepomucen/ funky fluid': 'funky fluid',
    'funky fluid/chmielarnia': 'funky fluid',
    'ghost brewing/piwowarownia': 'ghost brewing',
    'golem' : 'browar golem',
    'grupa żywiec': 'zywiec (grupa żywiec - heineken)',
    'holba' : 'holba',
    'jabeerwocky/napomucen' : 'jabeerwocky',
    'jan olbrach' : 'jan olbracht',
    'jan olbracht / kuźnia piwowarów' : 'jan olbracht',
    'jan olbracht/kuźnia piwowarów' : 'jan olbracht',
    'jan olbracht / viking brewmaster challenge 2018' : 'jan olbracht',
    'jan olbracht browar rzemieślniczy' : 'jan olbracht', 
    'jan olbracht browar staromiejski' : 'jan olbracht',
    'jan olbracht rzemieślniczy' : 'jan olbracht',
    'karuzela & piwnygaraż' : 'karuzela', 
    'karuzela & profesja' : 'karuzela',
    'kopyra & widawa' : 'widawa',
    'krakonos (cz)' : 'krakonos',
    'krakonoš' : 'krakonos',
    'księżny młyn' : 'księży młyn',
    'lindemans kriek' : 'lindemans',
    'lindemans (be)' : 'lindemans',
    'litivel' : 'litovel',
    'malgarden' : 'maltgarden',
    'maltarden' : 'maltgarden',
    'maryensztad' : 'maryensztadt',
    'maryensztadt' : 'maryensztadt',
    'mini browar reden' : 'minibrowar reden', 
    'minibrowar reden chorzów': 'minibrowar reden',
    'monsters & przetwórnia chmielu' : 'browar monsters',
    'měšťanský pivovar polička' : 'mestansky pivovar', 
    'měšťanský pivovar v poličce' : 'mestansky pivovar',
    'olimp' : 'browar olimp',
    'pinta' : 'browar pinta',
    'pinta & oud beersel brewery' : 'browar pinta',
    'pinta miesiąca' : 'browar pinta',
    'profejsa' : 'profesja',
    'palatum' : 'browar palatum',
    'palatum / szałpiw' : 'browar palatum',
    "palatum/l'instant" : 'browar palatum',
    'palatum/piwne podziemie' : 'piwne podziemie',
    'pinta / nepomucen' : 'nepomucen',
    'pinta miesiąca' : 'browar pinta',
    'piuu0027 fizz' : "piu' fizz",
    'pivovar litovel' : 'litovel',
    'piwna stopa / bednary' : 'bednary',
    'piwne podziemie/same krafty' : 'piwne podziemie',
    'piwowarownia' : 'piwowarownia',
    'pohjala/fuerst wiacek' : 'pohjala',
    'lervig / pohjala' : 'pohjala',
    'povovar svijany' : 'pivovar svijany',
    'pracowania piwa' : 'pracownia piwa',
    'profesja/jabeerwocky' : 'profesja',
    'primator (cz)' : 'primator',
    'recraft' : 'recraft',
    'socho & pora na kraft' : 'socho',
    'spółdzielczy / imperator' : 'spółdzielczy',
    'spółdzielczy/funky fluid' : 'spółdzielczy',
    'spółdzielczy/ harpagan' : 'spółdzielczy',
    'spółdzielczy/piwoteka' : 'spółdzielczy',
    'stu mostów/brlo' : 'stu mostów',
    'stu mostów/whiplash' : 'stu mostów',
    'stu mostów/bristol brewing' : 'stu mostów',
    'stu mostów/finback' : 'stu mostów',
    'stu mostów/mad scientist' : 'stu mostów',
    'stu mostów/northern monk' : 'stu mostów',
    'szpunt / kufle i kapsle' : 'szpunt',
    'tenczynek' : 'browar tenczynek',
    'trzech kumpli - browar lotny' : 'trzech kumpli',
    'velkopopovicky' : 'velke popovice',
    '#spiż' : 'minibrowar spiz',
    'jana' : 'browar jana',
    'fortuna' : 'browar fortuna',
    'czechy' : 'holba',
    'bury' : 'browar bury',
    'rockmill' : 'browar rockmill',
    'wilk' : 'browar wilk',
    'dear bear / szynkarnia' : 'deer bear'
}

In [4]:
ontap_data = list()

driver = webdriver.Chrome(r"chromedriver.exe")
driver.get('https://ontap.pl/')
driver.find_element_by_css_selector('#over18 > div > div > div.modal-body > button.btn.btn-lg.btn-success.pull-right').click()
cities = driver.find_elements_by_css_selector('#pubs > a')

for city_no in range(len(cities)):
    city = driver.find_elements_by_css_selector('#pubs > a')[city_no]
    city_name = city.text.split(' ')[0]
    city.click()
    pubs_in_city = driver.find_elements_by_css_selector('body > div.container-fluid > div.row > div')
    for pub_no in range(len(pubs_in_city)):
        pub = driver.find_elements_by_css_selector('body > div.container-fluid > div.row > div')[pub_no]
        pub.click()
        
        pub_name = driver.find_element_by_css_selector('body > div.container-fluid > div:nth-child(5) > div > address > strong').text.split('\n')[0]
        logger_scrapping.info(f'{datetime.datetime.now()}|ontap_scrapped - pub {pub_name}')
        
        beers_in_pub = driver.find_elements_by_css_selector('body > div.container-fluid > div:nth-child(7) > div > div')
        for beer in beers_in_pub:
            try:
                info = beer.find_element_by_class_name('cml_semi')
            except:
                continue
                
            if info.find_element_by_css_selector('h4 > span').text.split('\n')[1] != "Brak":
                tap_no = info.find_element_by_css_selector('h5').text
                brewery = info.find_element_by_css_selector('h4 > span').text.split('\n')[0].strip()
                beer_name = info.find_element_by_css_selector('h4 > span').text.split('\n')[1].strip()
                plato_abv = info.find_element_by_css_selector('h4 > span').text.split('\n')[2].strip()
                ontap_since = info.find_element_by_css_selector('div > span > span').text
                style = info.find_element_by_css_selector('div > span > b').text
                footer = beer.find_element_by_class_name('panel-footer')
                prices = footer.find_element_by_class_name('col-xs-7').text                
                portals_rates = footer.find_element_by_class_name('col-xs-5').text

                ontap_data.append(
                {
                    'city' : city_name,
                    'pub_name' : pub_name,
                    'brewery': brewery,
                    'beer_name' : beer_name,
                    'plato_abv' : plato_abv,
                    'on_tap_since' : ontap_since,
                    'style' : style,
                    'prices' : prices,
                    'portals_rates' : portals_rates,
                })
            
        driver.execute_script("window.history.go(-1)")
        time.sleep(1)
    driver.execute_script("window.history.go(-1)")
    time.sleep(1)
driver.close()

### save scrapped data as json file

In [5]:
import json
with open(f'ontap_data.json', 'w') as f:
    json.dump(ontap_data, f)

# Exploratory Data Analysis

### import json to pandas data frame

In [6]:
import pandas as pd
df = pd.read_json('ontap_data.json')

# lower all strings

In [7]:
df['beer_name'] = df['beer_name'].str.lower()
df['brewery'] = df['brewery'].str.lower()
df['city'] = df['city'].str.lower()
df['pub_name'] = df['pub_name'].str.lower()

# use dictionaries to correct wrong feature values

In [9]:
def cut_breweries_name(brewery):
    if brewery.lower()[:7] == 'browar ':
        return brewery[7:]
    else:
        return brewery

df['brewery'] = df['brewery'].apply(cut_breweries_name)
df['brewery'] = df['brewery'].replace(breweries)
df['beer_name'] = df['beer_name'].replace(
    {
        'pan i pani double':'pan ipani double',
        'panipani':'pan ipani',
        'double pan ipani':'pan ipani double',
        'art+ 8':'art8',
        'art+ 24':'art24',
        'art+ 9':'art9',
        'salamander raspberry & peach gose':'Raspberry and Peach Gose',
        'salamander raspberry u0026 peach gose':'Raspberry and Peach Gose',
        'art+ 28 amarillo & mosaic':'art28',
        'art+ 22':'art22',
        'pierwsza pomoc 10,5°':'pierwsza pomoc',
        'riki tiki - pinta miesiąca':'riki tiki',
        'atak chmielu 15,1°':'atak chmielu',
        'imperator bałtycki rum & bourbon ba with vanilla':'imperator bałtycki rum & bourbon ba',
        'hoplaaga 15,5°':'hoplaaga',
        'viva la wita! 16,5°':'viva la wita!',
        'apetyt na życie 13,1°':'apetyt na życie',
        'imperator bałtycki rum&bourbon with vanilla':'imperator bałtycki rum & bourbon ba',
        'modern drinking 15,5°':'modern drinking',
        'sanrajza 13,1°' : 'sanrajza',
        'oto mata ipa 14,0°' : 'oto mata ipa',
        'a nasasy?' : 'a nanasy?',
        'jean hop vam damm' : 'jean-hop vam damm',
        'al apacino zest ed.' : 'al apacino',
        'michaił jakson' : 'michail jakson',
        'michail jackson' : 'michail jakson',
        'miroslaw gose' : 'miroslav gose',
        'cud na wisłą' : 'cud nad wisłą',
        'jelonki lgbt' : 'jelonki',
        'coś na wieczór ?' : 'coś na wieczór?',
        "małe czy duże ?" : 'małe czy duże?',
        'ciężki dzień' : 'ciezki dzien',
        'prosty pils' : 'pils',
        'lab-25' : 'lab 25',
        'thaddeaus vel tadzik' : 'theoddeus vel tadzik',
        'dark premium' : 'premium dark',
        'blend battle vol. 1' : 'blend battle vol.1',
        'polski krafcik vol.4' : 'polski krafcik vol. 4',
        "let's cook - apricot&lime gose" : "let's cook - gose z morelą i limonką",
        "let's cook mango" : "cook - mango",
        "let's cook - mango z gose" : "cook - mango",
        "let's cook apricot & lime gose" : "cook - gose z morelą i limonką",
        "let's cook - gose z morelą i limonką" : "cook - gose z morelą i limonką",
        "gose z porzeczką": "gose z porzeczkami",
        'king of hope' : 'king of hop',
        'ris in peace autumn' : 'RIS in Peace - Autumn',
        'barrel aged projekt double dry hopped new zealand ipa white wine' : 'barrel aged ddh new zealand ipa White Wine',
        'hoptime apa' : 'hoptime american pale ale',
        'sourtime blackberry ipa' : 'sourtime blackberry',
        'żywot barleya' : "żywot barley'a",
        'sourtime marakuja new england sour ipa' : 'sourtime marakuja',
        'blackcurrant sour' : 'black currant',
        'apricot wheat' : 'funky fruit apricot',
        'poly pay' : 'polypay',
        'sokowirówka - jeżyna i jagoda' : 'sokowirówka leśne',
        'kozel svetly' : 'kozel světlý',
        'ciemna strona cieszyna' : 'ciemna strona',
        'raciborskie klasyczne' : 'raciborskie klasycznie warzone',
        'risnieft - rok leżakowany' : 'risnieft',
        'rich n creamy' : "rich 'n creamy",
        'coffelicious specjal' : 'coffeelicious special',
        'tropicalia' : 'tropikalia',
        'lemoniada mango/ananas' : 'lemoniada mango / ananas',
        "let's get underessed" : "let's get undressed",
        'hopsbant milkshake peach' : 'hopsbant peach milkshake',
        'birbant klasyczny: american pale ale' : 'klasyczny american pale ale',
        'hopsbant peach milkshake ipa' : 'hopsbant peach milkshake',
        'impeial citra' : 'imperial citra',
        'impeial citra ipa' : 'imperial citra',
        'dasz is kwass' : 'das is kwass',
        'piernikowe' : 'piernikowy foch',
        'trip-hop 6' : 'trip-hop 06',
        'trip-hop 2' : 'trip-hop sabro citra',
        'chill lemon summer ale' : 'chill',
        'skonana bufetowa 2019' : 'skonana bufetowa',
        "mr. hard's rocks milk 2018" : "mr. hard's rocks milk",
        'bo.hater' : 'bo*hater',
        'apa.rat' : 'apa*rat',
        'drunk.ula' : 'drunk*ula',
        'st.out' : 'st*out',
        'primator' : 'primátor premium lager 12°',
        'primator stout' : 'primátor stout',
        'primator weizen' : 'primátor weizen',
        'primator wizenbier' : 'primátor weizen',
        'primator weizenbier' : 'primátor weizen',
        'premium dark' : 'dark lager',
        'double dybuk bourbon 4r barrel aged': 'double dybuk 4r barrel aged',
        'double dybuk bourbon (vintage 2018) 4r': 'double dybuk 4r barrel aged',
        'golem vs vampire' : 'golem vs. vampire',
        'golem vs piranhas' : 'golem vs. piranhas',
        'acidum fructum 3': 'acidum fructum #3',
        'acidum fructum 2' : 'acidum fructum #2',
        'blend battle vol.1' : 'blend battle vol. 1',
        'cerny citron' : 'citron',
        'spiż - jasne' : 'jasne',
        'spiż-truskawkowe' : 'truskawkowy',
        'spiż - karmelowe' : 'karmelowe',
        'spiż - miodowe' : 'miodowe',
        'spiż - pszeniczne' : 'pszeniczne',
        'spiż - aipa' : 'aipa',
        'spiż - wiśniowe' : 'wiśniowe',
        'spiż -ciemne' : 'ciemne',
        'hopsztos' : 'hop sztos',
        'kwas ice tea' : 'kwassi ice tea',
        'imperial żyrardomyces wild ba' : 'żyrardomyces wild imperial',
        'jurajska pomarańcza' : 'ju-rajska pomarańcza',
        'such a łan' : 'such a lan',
        'such a dzimi' : 'such a dżimi',
        'such a american' : 'such an american',
        'primator - weizenbier' : 'tor weizen',
        'primátor weizenbier' : 'tor weizen',
        'holba šerák 11°' : 'holba šerák 11',
        'holba serak' : 'holba šerák 11',
        'holba' : 'holba premium 12',
        'premium' : 'holba premium 12',
        'aipa' : 'podroze kormorana american ipa',
        'podróże kormorana witbier' : 'podroze kormorana witbier',
        'wiśnia w piwie' : 'wisnia w piwie',
        'imperium kara mustfy' : 'imperium kara mustafy',
        'fruit bomb ipa' : 'fruit bomb! ipa',
        'augustiańkie' : 'augustianskie',
        'polish hops greatest hits' : 'polish hops greatest hitc',
        'tropical el dorado ipa' : 'tropical eldorado ipa',
        "buddha's hand lemon gose" : 'hand lemon gose'
    }
)

# Get ratebeer average score directly from ratebeer.com, what's more get beer style, because it looks like style at ontap is being filled in manually

In [10]:
df['rb_style'] = np.NaN

In [13]:
driver = webdriver.Chrome(r"chromedriver.exe")
for brewery in df[(df['rb_style'].isna())]['brewery'].value_counts().sort_values(ascending=False).index:
    driver.get(f'https://www.ratebeer.com/search?q={brewery}')
    driver.find_element_by_xpath(r"//*[contains(text(), 'Brewers')]").click()
    time.sleep(3)
    try:
        driver.find_element_by_xpath(r'//*[@id="root"]/div/div[2]/div/div/div[1]/div[2]/div[2]/a/div[1]/div/div[1]').click() #click first brewery from list
        time.sleep(3)
        for beer_name in df[(df['brewery'] == brewery) & (df['rb_style'].isna())]['beer_name'].unique():
            try:
                try:
                    driver.find_element_by_xpath(r"//*[contains(text(), 'beers found')]").click()
                except:
                    pass
                
                beer_page_obj = driver.find_element_by_xpath(f"//a[contains(translate(text(), '{beer_name.upper()}', '{beer_name.lower()}'), '{beer_name.lower()}')]/../../..")
                df.loc[(df['brewery'] == brewery) & (df['beer_name'] == beer_name), 'rb_style'] = beer_page_obj.find_element_by_css_selector('td:nth-child(1) > a').text
                df.loc[(df['brewery'] == brewery) & (df['beer_name'] == beer_name), 'rb_abv'] = beer_page_obj.find_element_by_css_selector('td:nth-child(2)').text
                df.loc[(df['brewery'] == brewery) & (df['beer_name'] == beer_name), 'rb_score'] = beer_page_obj.find_element_by_css_selector('td:nth-child(5)').text
                df.loc[(df['brewery'] == brewery) & (df['beer_name'] == beer_name), 'rb_style_perc'] = beer_page_obj.find_element_by_css_selector('td:nth-child(6)').text
                df.loc[(df['brewery'] == brewery) & (df['beer_name'] == beer_name), 'rb_reviews_no'] = beer_page_obj.find_element_by_css_selector('td:nth-child(7)').text
            except:
                counter = len(df[(df['brewery'] == brewery) & (df['beer_name'] == beer_name)]['beer_name'])
                logger_scrapping.warning(f'{datetime.datetime.now()}|error beer|{brewery}|{beer_name}|{counter}')
    except:
        counter = len(df[(df['brewery'] == brewery) & (df['rb_style'].isna())])
        logger_scrapping.warning(f'{datetime.datetime.now()}|error brewery|{brewery}|{counter}')
        
driver.close()

In [14]:
for rb_float in ['rb_abv', 'rb_score', 'rb_style_perc', 'rb_reviews_no']:
    df[rb_float] = df[rb_float].replace(['-', ''], np.NaN).replace('', np.NaN)
    df[rb_float] = df[rb_float].astype(float)

In [15]:
df.to_json('ontap_data_with_rb_scrapped.json')